### Word Group In Youtube

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Spanish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # native word end index
unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
word_usage_max = 100

# sentence ratio and time shift
adjust_sent_word_ratio = 30
adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [6]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [7]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [8]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [9]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [10]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
39995,greatest,253
39996,jotunheim,253
39997,schuurt,253
39998,ontvangstcomit,253


In [ ]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [ ]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [ ]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

39988

#### Youtube Sentence Analysis

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:10.760,00:00:13.200,jongens hier ben ik dan,WA8MV8nX9s0
1,00:00:13.200,00:00:17.040,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0
2,00:00:17.040,00:00:19.320,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0
3,00:00:19.900,00:00:23.760,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0
4,00:00:23.760,00:00:25.860,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0
...,...,...,...,...
639627,00:01:35.740,00:01:39.040,de link staat in de beschrijving van deze video,qzw2pblwjYU
639628,00:01:39.040,00:01:43.970,en als u ook de volgende afleveringen van zapp...,qzw2pblwjYU
639629,00:01:43.970,00:01:48.920,op ons kanaal en druk ook op het belletje er l...,qzw2pblwjYU
639630,00:01:48.920,00:01:51.580,zodra er weer een nieuwe video gepost wordt,qzw2pblwjYU


In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,10.76,13.20,jongens hier ben ik dan,WA8MV8nX9s0
1,1,13.20,17.04,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0
2,2,17.04,19.32,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0
3,3,19.90,23.76,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0
4,4,23.76,25.86,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0
...,...,...,...,...,...
639627,639627,95.74,99.04,de link staat in de beschrijving van deze video,qzw2pblwjYU
639628,639628,99.04,103.97,en als u ook de volgende afleveringen van zapp...,qzw2pblwjYU
639629,639629,103.97,108.92,op ons kanaal en druk ook op het belletje er l...,qzw2pblwjYU
639630,639630,108.92,111.58,zodra er weer een nieuwe video gepost wordt,qzw2pblwjYU


In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [ ]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,10.76,13.20,jongens hier ben ik dan,WA8MV8nX9s0,100.1,,"dan, ik, hier, ben, jongens"
1,1,13.20,17.04,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0,91.8,,"geks, maar, gaan, ook, dan, iets, en, mee, mak..."
2,2,17.04,19.32,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0,100.1,,"bedoel, gaan, ik, de, naar, royal, bridge, sui..."
3,3,19.90,23.76,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0,100.1,,"bedenken, maar, meest, kan, luxe, de, is, je, ..."
4,4,23.76,25.86,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0,100.1,,"bekende, mensen, meest, hebben, hier, de, gesl..."
...,...,...,...,...,...,...,...,...
639627,639627,95.74,99.04,de link staat in de beschrijving van deze video,qzw2pblwjYU,89.0,,"in, link, deze, staat, de, video, beschrijving..."
639628,639628,99.04,103.97,en als u ook de volgende afleveringen van zapp...,qzw2pblwjYU,82.5,"abonneren, zappi","ook, zien, en, als, kunt, best, volgende, de, ..."
639629,639629,103.97,108.92,op ons kanaal en druk ook op het belletje er l...,qzw2pblwjYU,94.2,,"op, dan, belletje, ook, druk, en, ons, direct,..."
639630,639630,108.92,111.58,zodra er weer een nieuwe video gepost wordt,qzw2pblwjYU,100.1,,"wordt, gepost, weer, zodra, nieuwe, video, er,..."


In [ ]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,10.76,13.20,jongens hier ben ik dan,WA8MV8nX9s0,100.1,NaN,"dan, ik, hier, ben, jongens"
1,1,13.20,17.04,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0,91.8,NaN,"geks, maar, gaan, ook, dan, iets, en, mee, mak..."
2,2,17.04,19.32,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0,100.1,NaN,"bedoel, gaan, ik, de, naar, royal, bridge, sui..."
3,3,19.90,23.76,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0,100.1,NaN,"bedenken, maar, meest, kan, luxe, de, is, je, ..."
4,4,23.76,25.86,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0,100.1,NaN,"bekende, mensen, meest, hebben, hier, de, gesl..."
...,...,...,...,...,...,...,...,...
639627,639627,95.74,99.04,de link staat in de beschrijving van deze video,qzw2pblwjYU,89.0,NaN,"in, link, deze, staat, de, video, beschrijving..."
639628,639628,99.04,103.97,en als u ook de volgende afleveringen van zapp...,qzw2pblwjYU,82.5,"abonneren, zappi","ook, zien, en, als, kunt, best, volgende, de, ..."
639629,639629,103.97,108.92,op ons kanaal en druk ook op het belletje er l...,qzw2pblwjYU,94.2,NaN,"op, dan, belletje, ook, druk, en, ons, direct,..."
639630,639630,108.92,111.58,zodra er weer een nieuwe video gepost wordt,qzw2pblwjYU,100.1,NaN,"wordt, gepost, weer, zodra, nieuwe, video, er,..."


In [ ]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,10.76,13.20,jongens hier ben ik dan,WA8MV8nX9s0,100.1,NaN,"dan, ik, hier, ben, jongens"
1,1,13.20,17.04,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0,91.8,NaN,"geks, maar, gaan, ook, dan, iets, en, mee, mak..."
2,2,17.04,19.32,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0,100.1,NaN,"bedoel, gaan, ik, de, naar, royal, bridge, sui..."
3,3,19.90,23.76,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0,100.1,NaN,"bedenken, maar, meest, kan, luxe, de, is, je, ..."
4,4,23.76,25.86,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0,100.1,NaN,"bekende, mensen, meest, hebben, hier, de, gesl..."
...,...,...,...,...,...,...,...,...
634977,639627,95.74,99.04,de link staat in de beschrijving van deze video,qzw2pblwjYU,89.0,NaN,"in, link, deze, staat, de, video, beschrijving..."
634978,639628,99.04,103.97,en als u ook de volgende afleveringen van zapp...,qzw2pblwjYU,82.5,"abonneren, zappi","ook, zien, en, als, kunt, best, volgende, de, ..."
634979,639629,103.97,108.92,op ons kanaal en druk ook op het belletje er l...,qzw2pblwjYU,94.2,NaN,"op, dan, belletje, ook, druk, en, ons, direct,..."
634980,639630,108.92,111.58,zodra er weer een nieuwe video gepost wordt,qzw2pblwjYU,100.1,NaN,"wordt, gepost, weer, zodra, nieuwe, video, er,..."


In [ ]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
0,0,10.76,13.20,jongens hier ben ik dan,WA8MV8nX9s0,100.1,NaN,"dan, ik, hier, ben, jongens",jongens hier ben ik dan
1,1,13.20,17.04,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0,91.8,NaN,"geks, maar, gaan, ook, dan, iets, en, mee, mak...",en we gaan iets heel maar dan ook heel geks me...
2,2,17.04,19.32,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0,100.1,NaN,"bedoel, gaan, ik, de, naar, royal, bridge, sui...",ik bedoel we gaan naar de royal bridge suite
3,3,19.90,23.76,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0,100.1,NaN,"bedenken, maar, meest, kan, luxe, de, is, je, ...",dit is de meest luxe suite die je maar kan bed...
4,4,23.76,25.86,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0,100.1,NaN,"bekende, mensen, meest, hebben, hier, de, gesl...",de meest bekende mensen hebben hier geslapen
...,...,...,...,...,...,...,...,...,...
634977,639627,95.74,99.04,de link staat in de beschrijving van deze video,qzw2pblwjYU,89.0,NaN,"in, link, deze, staat, de, video, beschrijving...",de link staat in de beschrijving van deze video
634978,639628,99.04,103.97,en als u ook de volgende afleveringen van zapp...,qzw2pblwjYU,82.5,"abonneren, zappi","ook, zien, en, als, kunt, best, volgende, de, ...",en als u ook de volgende afleveringen van
634979,639629,103.97,108.92,op ons kanaal en druk ook op het belletje er l...,qzw2pblwjYU,94.2,NaN,"op, dan, belletje, ook, druk, en, ons, direct,...",op ons kanaal en druk ook op het belletje er l...
634980,639630,108.92,111.58,zodra er weer een nieuwe video gepost wordt,qzw2pblwjYU,100.1,NaN,"wordt, gepost, weer, zodra, nieuwe, video, er,...",zodra er weer een nieuwe video gepost wordt


In [ ]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,jongens hier ben ik dan,10.76,13.200000,jongens hier ben ik dan,WA8MV8nX9s0
1,en we gaan iets heel maar dan ook heel geks me...,13.20,17.040000,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0
2,ik bedoel we gaan naar de royal bridge suite,17.04,19.320000,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0
3,dit is de meest luxe suite die je maar kan bed...,19.90,23.760000,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0
4,de meest bekende mensen hebben hier geslapen,23.76,25.860000,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0
...,...,...,...,...,...
500375,de link staat in de beschrijving van deze video,95.74,99.040000,de link staat in de beschrijving van deze video,qzw2pblwjYU
500376,en als u ook de volgende afleveringen van,99.04,101.392955,en als u ook de volgende afleveringen van zapp...,qzw2pblwjYU
500377,op ons kanaal en druk ook op het belletje er l...,103.97,108.920000,op ons kanaal en druk ook op het belletje er l...,qzw2pblwjYU
500378,zodra er weer een nieuwe video gepost wordt,108.92,111.580000,zodra er weer een nieuwe video gepost wordt,qzw2pblwjYU


In [ ]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,jongens hier ben ik dan,10.46,13.500000,jongens hier ben ik dan,WA8MV8nX9s0
1,en we gaan iets heel maar dan ook heel geks me...,12.90,17.340000,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0
2,ik bedoel we gaan naar de royal bridge suite,16.74,19.620000,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0
3,dit is de meest luxe suite die je maar kan bed...,19.60,24.060000,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0
4,de meest bekende mensen hebben hier geslapen,23.46,26.160000,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0
...,...,...,...,...,...
500375,de link staat in de beschrijving van deze video,95.44,99.340000,de link staat in de beschrijving van deze video,qzw2pblwjYU
500376,en als u ook de volgende afleveringen van,98.74,101.692955,en als u ook de volgende afleveringen van zapp...,qzw2pblwjYU
500377,op ons kanaal en druk ook op het belletje er l...,103.67,109.220000,op ons kanaal en druk ook op het belletje er l...,qzw2pblwjYU
500378,zodra er weer een nieuwe video gepost wordt,108.62,111.880000,zodra er weer een nieuwe video gepost wordt,qzw2pblwjYU


In [ ]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,jongens hier ben ik dan,10,14,jongens hier ben ik dan,WA8MV8nX9s0
1,en we gaan iets heel maar dan ook heel geks me...,13,17,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0
2,ik bedoel we gaan naar de royal bridge suite,17,20,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0
3,dit is de meest luxe suite die je maar kan bed...,20,24,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0
4,de meest bekende mensen hebben hier geslapen,23,26,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0
...,...,...,...,...,...
500375,de link staat in de beschrijving van deze video,95,99,de link staat in de beschrijving van deze video,qzw2pblwjYU
500376,en als u ook de volgende afleveringen van,99,102,en als u ook de volgende afleveringen van zapp...,qzw2pblwjYU
500377,op ons kanaal en druk ook op het belletje er l...,104,109,op ons kanaal en druk ook op het belletje er l...,qzw2pblwjYU
500378,zodra er weer een nieuwe video gepost wordt,109,112,zodra er weer een nieuwe video gepost wordt,qzw2pblwjYU


In [ ]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,het vogeltje komt al dichterbij en je vliegt e...,1135,1171,het vogeltje komt al dichterbij en je vliegt e...,iJb2QNlOkpY
1,maar jij bent zo moe je hoort het bijna niet m...,1591,1650,maar jij bent zo moe je hoort het bijna niet m...,bNmlhWkM4GE
2,ik hoop in de toekomst gewoon dat het beter ga...,1122,1143,ik hoop in de toekomst gewoon dat het beter ga...,GG9RNnB1yy8
3,en de vlinders zijn mooi van kleur je kijkt er...,295,323,en de vlinders zijn mooi van kleur je kijkt er...,iJb2QNlOkpY
4,hier slapen alle dieren in rust en wij gaan de...,1251,1277,hier slapen alle dieren in rust en wij gaan de...,iJb2QNlOkpY
...,...,...,...,...,...
500375,de i en de j,88,90,de i en de j,ADGaSMuaVbc
500376,de u ja de h,1678,1681,de u ja de h,ZdQ9EstSpig
500377,jo l is jo l,308,309,jo l is jo l yasser is yasser en charlie is ch...,NQ4fc65Q8VQ
500378,dus l h l h,228,233,dus l h l h,BwfFVQLRkYM


In [ ]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,het vogeltje komt al dichterbij en je vliegt e...,1135,1171,het vogeltje komt al dichterbij en je vliegt e...,iJb2QNlOkpY,75
1,maar jij bent zo moe je hoort het bijna niet m...,1591,1650,maar jij bent zo moe je hoort het bijna niet m...,bNmlhWkM4GE,65
2,ik hoop in de toekomst gewoon dat het beter ga...,1122,1143,ik hoop in de toekomst gewoon dat het beter ga...,GG9RNnB1yy8,73
3,en de vlinders zijn mooi van kleur je kijkt er...,295,323,en de vlinders zijn mooi van kleur je kijkt er...,iJb2QNlOkpY,59
4,hier slapen alle dieren in rust en wij gaan de...,1251,1277,hier slapen alle dieren in rust en wij gaan de...,iJb2QNlOkpY,73
...,...,...,...,...,...,...
500375,de i en de j,88,90,de i en de j,ADGaSMuaVbc,80
500376,de u ja de h,1678,1681,de u ja de h,ZdQ9EstSpig,80
500377,jo l is jo l,308,309,jo l is jo l yasser is yasser en charlie is ch...,NQ4fc65Q8VQ,60
500378,dus l h l h,228,233,dus l h l h,BwfFVQLRkYM,60


In [ ]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id
0,het zijn geen net zoals met de voice of wat wi...,517,529,het zijn geen net zoals met de voice of wat wi...,bBYvUrZhWXs
1,dus als het gaat om de slecht nieuws gesprekke...,1110,1121,dus als het gaat om de slecht nieuws gesprekke...,QPALQDtz7so
2,op moment van bedenken is het blijkbaar niet z...,333,347,op moment van bedenken is het blijkbaar niet z...,xKhL2kGvszM
3,het is inderdaad mogelijk om bij elkaar te bli...,261,270,het is inderdaad mogelijk om bij elkaar te bli...,7D5ZiQsUy48
4,en afhankelijk van waar je woont in nederland ...,1103,1112,en afhankelijk van waar je woont in nederland ...,gQPOBsiqB_o
...,...,...,...,...,...
374977,ik ga nu de co,122,126,ik ga nu de co,_yzabW5dGr4
374978,is de c in ch,1190,1199,is de c in ch,WB2jYe9y_Lk
374979,est d di e au,2731,2733,la premi re semaine est d di e au suivi inform...,bqqC2uPC2q8
374980,t is maar n h,534,538,t is maar n h,ysO6ouvo4W0


In [ ]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-104-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,het zijn geen net zoals met de voice of wat wi...,517,529,het zijn geen net zoals met de voice of wat wi...,bBYvUrZhWXs,100
1,dus als het gaat om de slecht nieuws gesprekke...,1110,1121,dus als het gaat om de slecht nieuws gesprekke...,QPALQDtz7so,100
2,op moment van bedenken is het blijkbaar niet z...,333,347,op moment van bedenken is het blijkbaar niet z...,xKhL2kGvszM,100
3,het is inderdaad mogelijk om bij elkaar te bli...,261,270,het is inderdaad mogelijk om bij elkaar te bli...,7D5ZiQsUy48,100
4,en afhankelijk van waar je woont in nederland ...,1103,1112,en afhankelijk van waar je woont in nederland ...,gQPOBsiqB_o,100
...,...,...,...,...,...,...
374977,ik ga nu de co,122,126,ik ga nu de co,_yzabW5dGr4,100
374978,is de c in ch,1190,1199,is de c in ch,WB2jYe9y_Lk,100
374979,est d di e au,2731,2733,la premi re semaine est d di e au suivi inform...,bqqC2uPC2q8,17
374980,t is maar n h,534,538,t is maar n h,ysO6ouvo4W0,100


In [ ]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id
0,het zijn geen net zoals met de voice of wat wi...,517,529,het zijn geen net zoals met de voice of wat wi...,bBYvUrZhWXs
1,dus als het gaat om de slecht nieuws gesprekke...,1110,1121,dus als het gaat om de slecht nieuws gesprekke...,QPALQDtz7so
2,op moment van bedenken is het blijkbaar niet z...,333,347,op moment van bedenken is het blijkbaar niet z...,xKhL2kGvszM
3,het is inderdaad mogelijk om bij elkaar te bli...,261,270,het is inderdaad mogelijk om bij elkaar te bli...,7D5ZiQsUy48
4,en afhankelijk van waar je woont in nederland ...,1103,1112,en afhankelijk van waar je woont in nederland ...,gQPOBsiqB_o
...,...,...,...,...,...
247067,m r er is meer,275,277,m r er is meer,0f2WmZuMNGY
247068,oh en op is ie,163,165,oh en op is ie,EKGFddIaTUQ
247069,ik ga nu de co,122,126,ik ga nu de co,_yzabW5dGr4
247070,is de c in ch,1190,1199,is de c in ch,WB2jYe9y_Lk


In [ ]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,veel mensen weten niks over hun eigen identiteit,zzF0BXS9h_0
1,specificaties kijken want daar staat namelijk ...,zrh3pdVEd6A
2,want hij laat nou eenmaal minder licht binnen,zqFScxYMMFQ
3,bolletje krokante ontbijtgranen bevatten allee...,zpJutGvF6QM
4,bijzondere eigenschappen zijn vaak van generat...,zmvPgBfIQns
...,...,...
2607,ja zij gaat nu natuurlijk chanel bellen,05uOcgS2jn8
2608,iemand anders neerhalen because they do,05b5ALhkQ9w
2609,minimaal zes uur per dag,03UN78UHU4w
2610,twitter was toen net helemaal hot,02MtK2r49UI


In [ ]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,het zijn geen net zoals met de voice of wat wi...,517,529,het zijn geen net zoals met de voice of wat wi...,bBYvUrZhWXs
1,dus als het gaat om de slecht nieuws gesprekke...,1110,1121,dus als het gaat om de slecht nieuws gesprekke...,QPALQDtz7so
2,op moment van bedenken is het blijkbaar niet z...,333,347,op moment van bedenken is het blijkbaar niet z...,xKhL2kGvszM
3,het is inderdaad mogelijk om bij elkaar te bli...,261,270,het is inderdaad mogelijk om bij elkaar te bli...,7D5ZiQsUy48
4,en afhankelijk van waar je woont in nederland ...,1103,1112,en afhankelijk van waar je woont in nederland ...,gQPOBsiqB_o
...,...,...,...,...,...
2569,i just put it on,697,700,i just put it on,y3_ZGouZ0xg
2570,its a bit to far,1034,1036,its a bit to far,MZBZFbL_vOQ
2571,for now i do not,267,274,for now i do not,82v1HAy9ckI
2572,hoe kan iets w l,1501,1503,hoe kan iets w l,67KX1u7gfIo


In [ ]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,het zijn geen net zoals met de voice of wat wi...,517,529,het zijn geen net zoals met de voice of wat wi...,bBYvUrZhWXs,https://www.youtube.com/watch?v=bBYvUrZhWXs&t=...
1,dus als het gaat om de slecht nieuws gesprekke...,1110,1121,dus als het gaat om de slecht nieuws gesprekke...,QPALQDtz7so,https://www.youtube.com/watch?v=QPALQDtz7so&t=...
2,op moment van bedenken is het blijkbaar niet z...,333,347,op moment van bedenken is het blijkbaar niet z...,xKhL2kGvszM,https://www.youtube.com/watch?v=xKhL2kGvszM&t=...
3,het is inderdaad mogelijk om bij elkaar te bli...,261,270,het is inderdaad mogelijk om bij elkaar te bli...,7D5ZiQsUy48,https://www.youtube.com/watch?v=7D5ZiQsUy48&t=...
4,en afhankelijk van waar je woont in nederland ...,1103,1112,en afhankelijk van waar je woont in nederland ...,gQPOBsiqB_o,https://www.youtube.com/watch?v=gQPOBsiqB_o&t=...
...,...,...,...,...,...,...
2569,i just put it on,697,700,i just put it on,y3_ZGouZ0xg,https://www.youtube.com/watch?v=y3_ZGouZ0xg&t=...
2570,its a bit to far,1034,1036,its a bit to far,MZBZFbL_vOQ,https://www.youtube.com/watch?v=MZBZFbL_vOQ&t=...
2571,for now i do not,267,274,for now i do not,82v1HAy9ckI,https://www.youtube.com/watch?v=82v1HAy9ckI&t=...
2572,hoe kan iets w l,1501,1503,hoe kan iets w l,67KX1u7gfIo,https://www.youtube.com/watch?v=67KX1u7gfIo&t=...


In [ ]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [ ]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [ ]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [112]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['Dutch_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv',
 'Dutch_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx',
 'Dutch_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv',
 'Dutch_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx',
 'Dutch_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv',
 'Dutch_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx',
 'Dutch_Word_Group_Max_In_Youtube_Sentence_5000_Word6.csv',
 'Dutch_Word_Group_Max_In_Youtube_Sentence_5000_Word6.xlsx',
 'Dutch_Word_Group_Max_In_Youtube_Sentence_10000_Word6.csv',
 'Dutch_Word_Group_Max_In_Youtube_Sentence_10000_Word6.xlsx',
 'Dutch_Word_Group_Max_In_Youtube_Sentence_20000_Word6.csv',
 'Dutch_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx',
 'Dutch_Word_Group_Max_In_Youtube_Sentence_40000_Word6.csv',
 'Dutch_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx']

In [113]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [114]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Youtube Videoid Text Analysis

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

In [ ]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

In [ ]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

In [ ]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

In [ ]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list2 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list2.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [ ]:
df_text_ratio_result = pd.DataFrame(list(result_list2), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

In [ ]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

In [ ]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

In [ ]:
if word_group_adjust_select:
    df_adjust_text_ratio = df_exract_word_group(df_adjust_text_ratio, "sentence", "video_id", word_list, sent_len=word_group_adjust_select, sent_len_num=sentence_word_count_number)
else:
    df_adjust_text_ratio.loc[:,"search_string"] = df_adjust_text_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
    df_adjust_text_ratio = df_adjust_text_ratio[~(df_adjust_text_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
    
df_adjust_text_ratio

In [ ]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

In [ ]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [ ]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

In [ ]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [ ]:
#df_word_group_time_loc_in_text_result.to_csv(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.csv", index=False)

In [ ]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Youtube Videoid Text Adjust Result Analysis Multiprocess

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 50
adjust_text_word_ratio = 5
shift = 0.3  # sentence time shift

# func select
word_group_adjust_select = True  # True, False; True for selecting word group according to sentence_word_count_number
sentence_word_count_number = 5   # False for selecting word group according to all word group length

# # shared word frequency
# shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

In [ ]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

In [ ]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [ ]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [ ]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

In [ ]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

In [ ]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

In [ ]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

In [ ]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list3 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list3.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [ ]:
df_text_ratio_result = pd.DataFrame(list(result_list3), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

In [ ]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result

In [ ]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list4 = manager.list()
index_list = manager.list()
all_index_list = manager.list()
var_index_list = manager.list()
text_all_list = manager.list()  
df = df_adjust_text_ratio
df.reset_index(drop=True, inplace=True)
index_num = list(range(len(df)))
sent_len = word_group_adjust_select
sent_len_num = sentence_word_count_number 

def df_exract_word_group(index_num):    
    source_text = df.loc[index_num,"sentence"]
    opt_var = df.loc[index_num,"video_id"]
    words = re.findall(r"\w+", source_text, re.UNICODE)
    
    index_list = []
    for j in range(len(words)):
        if words[j] in word_list:
            index_list.append(j)
        else:
            pass
    all_index_list = []
    var_index_list = []
    for k in range(len(index_list)):
        try:
            var1 = index_list[k] + 1  
            var2 = index_list[k+1]
        except:
            var1 = index_list[k] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[k]
            var_index_list.append(var3)
        else:
            var3 = index_list[k]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []
    text_all_list = []
    for m in all_index_list:
        text_list = [] 
        for n in m:
            word = words[n]
            text_list.append(word)
            if sent_len:
                if len(text_list) >= sent_len_num:
                    text = " ".join(text_list)
                    text_all_list.append(text)
                else:
                    pass
            else:
               text = " ".join(text_list)
               text_all_list.append(text) 
        
    for search_string in text_all_list:
        #df_var["index"] = i
        #df_var["search_string"] = search_string
        #df_var["sentence"] = source_text
        #df_var["video_id"] = opt_var
        result_list4.append([index_num,search_string,source_text,opt_var])

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(df_exract_word_group, index_num)

In [ ]:
df_adjust_text_ratio = pd.DataFrame(list(result_list4), columns=["index","search_string","sentence","video_id"])
df_adjust_text_ratio.sort_values(by="index", ascending=True, inplace=True)
df_adjust_text_ratio.reset_index(drop=True, inplace=True)
df_adjust_text_ratio.drop(["index"], axis=1, inplace=True)
df_adjust_text_ratio

In [ ]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

In [ ]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [ ]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

In [ ]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [ ]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_All_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

#### Temp

In [ ]:
df_28 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv")
df_28

In [ ]:
df_200 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv")
df_200

In [ ]:
df_1000 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv")
df_1000